In [28]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.utils import resample

In [29]:
mnist_train = pd.read_csv('mnist_train.csv')
mnist_test = pd.read_csv('mnist_test.csv')

In [30]:
# Define the number of samples desired for each class in each dataset
samples_per_class = 1000

# Initialize two empty DataFrames to store the selected samples for each dataset
selected_data_1 = pd.DataFrame()
selected_data_2 = pd.DataFrame()

# Iterate over each class label
for class_label in range(10):  # 10 classes for digits 0-9
    # Filter data for each class label
    class_data = mnist_train[mnist_train['label'] == class_label]
    
    # Select the first 150 samples for the first dataset
    first_samples = class_data.head(samples_per_class)
    selected_data_1 = pd.concat([selected_data_1, first_samples])
    
    # Select the next 150 samples for the second dataset
    next_samples = class_data.iloc[samples_per_class:samples_per_class*2]
    selected_data_2 = pd.concat([selected_data_2, next_samples])

# Resetting the index for both datasets
selected_data_1 = selected_data_1.reset_index(drop=True)
selected_data_2 = selected_data_2.reset_index(drop=True)
# shuffle the rows of both datasets
selected_data_1 = selected_data_1.sample(frac=1).reset_index(drop=True)
selected_data_2 = selected_data_2.sample(frac=1).reset_index(drop=True)


In [31]:
model_1_train = selected_data_1.drop('label', axis=1)
model_1_label = selected_data_1['label']
model_2_train = selected_data_2.drop('label', axis=1)
model_2_label = selected_data_2['label']
model_1_label = to_categorical(model_1_label,num_classes=10)
model_2_label = to_categorical(model_2_label,num_classes=10)

In [32]:
# Define a common neural network architecture
def create_neural_network(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [33]:
model_1=create_neural_network(model_1_train.shape[1], 10)
model_2=create_neural_network(model_2_train.shape[1], 10)

In [34]:
# Train the models
model_1.fit(model_1_train, model_1_label, epochs=30, batch_size=32, validation_split=0.2, verbose=0)
model_2.fit(model_2_train, model_2_label, epochs=30, batch_size=32, validation_split=0.2, verbose=0)

In [35]:
accuracy_1 = model_1.evaluate(mnist_test.drop('label', axis=1), to_categorical(mnist_test['label'], num_classes=10), verbose=0)[1]
accuracy_2 = model_2.evaluate(mnist_test.drop('label', axis=1), to_categorical(mnist_test['label'], num_classes=10), verbose=0)[1]
print('Accuracy of model 1: ', accuracy_1)
print('Accuracy of model 2: ', accuracy_2)

Accuracy of model 1:  0.9372000098228455
Accuracy of model 2:  0.9330999851226807


In [36]:
weights_1 = model_1.get_weights() 
weights_2 = model_2.get_weights()
alpha = 0.5
blended_weights = [alpha * w0 + (1 - alpha) * w1 for w0, w1 in zip(weights_1, weights_2)]
model_1.set_weights(blended_weights)
model_2.set_weights(blended_weights)

In [37]:
accuracy_1 = model_1.evaluate(mnist_test.drop('label', axis=1), to_categorical(mnist_test['label'], num_classes=10), verbose=0)[1]
accuracy_2 = model_2.evaluate(mnist_test.drop('label', axis=1), to_categorical(mnist_test['label'], num_classes=10), verbose=0)[1]
print('Accuracy of model 1: ', accuracy_1)
print('Accuracy of model 2: ', accuracy_2)

Accuracy of model 1:  0.36230000853538513
Accuracy of model 2:  0.36230000853538513
